# 계층형 분류기와 단일 분류기 성능 비교

동일한 데이터셋에 대해 학습한 계층형 분류 방식과 단일 분류 방식의 성능을 비교하고 분석합니다. 정확도, 정밀도, 재현율 등 다양한 평가지표를 사용하여 성능을 평가하고 결과를 비교합니다.

In [ ]:
FILE_PATH = 'assets/' 
MODEL_PATH = 'models/'

In [ ]:
# 서브레딧 및 그룹 정의
SUB_PER_GROUP = 4
SUBREDDITS = [
    'Thetruthishere', 'Glitch_in_the_Matrix', 'UnresolvedMysteries', 'Paranormal',
    'learnprogramming', 'cscareerquestions', 'SideProject', 'AskProgramming',
    'TrueFilm', 'booksuggestions', 'TrueGaming', 'LetsTalkMusic',
    'relationship_advice', 'AmItheAsshole', 'offmychest', 'Advice',
    'personalfinance', 'investing', 'Frugal', 'financialindependence',
]
GROUP_MAP = {
    'Mystery': SUBREDDITS[0:SUB_PER_GROUP], 
    'Dev': SUBREDDITS[SUB_PER_GROUP:2*SUB_PER_GROUP], 
    'Culture': SUBREDDITS[2*SUB_PER_GROUP:3*SUB_PER_GROUP], 
    'Life': SUBREDDITS[3*SUB_PER_GROUP:4*SUB_PER_GROUP],
    'Finance': SUBREDDITS[4*SUB_PER_GROUP:5*SUB_PER_GROUP],
}
VECTOR_DIMENSION = 5000  # 문서 벡터 차원

In [ ]:
import pandas as pd

# 데이터 로드
train_df = pd.read_csv(FILE_PATH + 'train_data.csv')
test_df = pd.read_csv(FILE_PATH + 'test_data.csv')

subreddit_df = train_df[train_df['subreddit'] == SUBREDDITS[0]].copy()
subreddit_df.head(5)

## 모델 로드

In [ ]:
# 모델 로드
import joblib

# 벡터라이저 로드
vectorizer = joblib.load(MODEL_PATH + 'tfidf_vectorizer.pkl')

# 상위 분류기 로드
top_classifier = joblib.load(MODEL_PATH + 'top_classifier.pkl')

# 하위 분류기 로드
sub_classifiers = {}
for group_name in GROUP_MAP.keys():
    sub_classifiers[group_name] = joblib.load(MODEL_PATH + f'sub_classifier_{group_name}.pkl')

# 단일 분류기 로드
single_classifier = joblib.load(MODEL_PATH + 'single_classifier.pkl')

## 성능 비교

In [ ]:
from sklearn.metrics import classification_report

def evaluate_hierarchical_classifier(X_test_vec, y_test_top, y_test_sub):
    # 상위 분류기 예측
    y_pred_top = top_classifier.predict(X_test_vec)
    
    # 하위 분류기 예측
    y_pred_sub = []
    for i, group in enumerate(y_pred_top):
        # 예측된 그룹에 맞는 하위 분류기 선택
        if group in sub_classifiers:
            sub_clf = sub_classifiers[group]
            # X_test_vec[i]는 1차원 배열이므로, (1, -1)로 reshape하여 2차원으로 만들어 예측
            sub_pred = sub_clf.predict(X_test_vec[i].reshape(1, -1))
            y_pred_sub.append(sub_pred[0])
        else:
            # 예외 처리: 학습되지 않은 그룹이 예측된 경우 (거의 발생 안 함)
            y_pred_sub.append("Unknown")
    
    # 성능 평가
    print("="*50)
    print("[계층형 분류기 성능 평가]")
    print(classification_report(y_test_sub, y_pred_sub))
    print("="*50)

# 단일 분류기 성능 평가
def evaluate_single_classifier(X_test_vec, y_test):
    # 단일 분류기 예측
    y_pred = single_classifier.predict(X_test_vec)
    
    # 성능 평가
    print("="*50)
    print("[단일 분류기 성능 평가]")
    print(classification_report(y_test, y_pred))
    print("="*50)

print("테스트 데이터 벡터화 진행 중...")
X_test_vec = vectorizer.transform(test_df['preprocessed_content']).toarray()

# 정답 레이블 준비 (Test 데이터 기준)
reverse_group_map = {subreddit: group for group, subreddits in GROUP_MAP.items() for subreddit in subreddits}

# 상위 그룹 정답 (Test)
y_test_group = test_df['subreddit'].map(reverse_group_map).tolist()

# 하위 서브레딧 정답 (Test)
y_test_sub = test_df['subreddit'].tolist()


# --- 평가 실행 ---

# 계층형 분류기 평가
evaluate_hierarchical_classifier(X_test_vec, y_test_group, y_test_sub)

# 단일 분류기 평가
evaluate_single_classifier(X_test_vec, y_test_sub)